# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8934498a60>
├── 'a' --> tensor([[-0.1392, -1.0507,  0.2461],
│                   [ 0.6169, -0.9462, -1.3576]])
└── 'x' --> <FastTreeValue 0x7f8934498c70>
    └── 'c' --> tensor([[-0.0786,  1.0145, -0.3751, -1.8855],
                        [ 1.2679,  0.6253,  0.5551, -0.4105],
                        [ 2.0189, -0.1684,  0.0351, -0.0676]])

In [4]:
t.a

tensor([[-0.1392, -1.0507,  0.2461],
        [ 0.6169, -0.9462, -1.3576]])

In [5]:
%timeit t.a

62.3 ns ± 0.0882 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8934498a60>
├── 'a' --> tensor([[-0.4606, -1.8415,  0.5770],
│                   [ 0.4975, -1.1773, -0.6797]])
└── 'x' --> <FastTreeValue 0x7f8934498c70>
    └── 'c' --> tensor([[-0.0786,  1.0145, -0.3751, -1.8855],
                        [ 1.2679,  0.6253,  0.5551, -0.4105],
                        [ 2.0189, -0.1684,  0.0351, -0.0676]])

In [7]:
%timeit t.a = new_value

70.5 ns ± 0.0457 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1392, -1.0507,  0.2461],
               [ 0.6169, -0.9462, -1.3576]]),
    x: Batch(
           c: tensor([[-0.0786,  1.0145, -0.3751, -1.8855],
                      [ 1.2679,  0.6253,  0.5551, -0.4105],
                      [ 2.0189, -0.1684,  0.0351, -0.0676]]),
       ),
)

In [10]:
b.a

tensor([[-0.1392, -1.0507,  0.2461],
        [ 0.6169, -0.9462, -1.3576]])

In [11]:
%timeit b.a

58.6 ns ± 0.0529 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9693,  0.4038,  1.5446],
               [ 0.8243, -0.1102,  0.0415]]),
    x: Batch(
           c: tensor([[-0.0786,  1.0145, -0.3751, -1.8855],
                      [ 1.2679,  0.6253,  0.5551, -0.4105],
                      [ 2.0189, -0.1684,  0.0351, -0.0676]]),
       ),
)

In [13]:
%timeit b.a = new_value

499 ns ± 0.681 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

852 ns ± 0.206 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 12 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 523 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 846 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f89344a0700>
├── 'a' --> tensor([[[-0.1392, -1.0507,  0.2461],
│                    [ 0.6169, -0.9462, -1.3576]],
│           
│                   [[-0.1392, -1.0507,  0.2461],
│                    [ 0.6169, -0.9462, -1.3576]],
│           
│                   [[-0.1392, -1.0507,  0.2461],
│                    [ 0.6169, -0.9462, -1.3576]],
│           
│                   [[-0.1392, -1.0507,  0.2461],
│                    [ 0.6169, -0.9462, -1.3576]],
│           
│                   [[-0.1392, -1.0507,  0.2461],
│                    [ 0.6169, -0.9462, -1.3576]],
│           
│                   [[-0.1392, -1.0507,  0.2461],
│                    [ 0.6169, -0.9462, -1.3576]],
│           
│                   [[-0.1392, -1.0507,  0.2461],
│                    [ 0.6169, -0.9462, -1.3576]],
│           
│                   [[-0.1392, -1.0507,  0.2461],
│                    [ 0.6169, -0.9462, -1.3576]]])
└── 'x' --> <FastTreeValue 0x7f8934498b20>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 30.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8934493340>
├── 'a' --> tensor([[-0.1392, -1.0507,  0.2461],
│                   [ 0.6169, -0.9462, -1.3576],
│                   [-0.1392, -1.0507,  0.2461],
│                   [ 0.6169, -0.9462, -1.3576],
│                   [-0.1392, -1.0507,  0.2461],
│                   [ 0.6169, -0.9462, -1.3576],
│                   [-0.1392, -1.0507,  0.2461],
│                   [ 0.6169, -0.9462, -1.3576],
│                   [-0.1392, -1.0507,  0.2461],
│                   [ 0.6169, -0.9462, -1.3576],
│                   [-0.1392, -1.0507,  0.2461],
│                   [ 0.6169, -0.9462, -1.3576],
│                   [-0.1392, -1.0507,  0.2461],
│                   [ 0.6169, -0.9462, -1.3576],
│                   [-0.1392, -1.0507,  0.2461],
│                   [ 0.6169, -0.9462, -1.3576]])
└── 'x' --> <FastTreeValue 0x7f8889214700>
    └── 'c' --> tensor([[-0.0786,  1.0145, -0.3751, -1.8855],
                        [ 1.2679,  0.6253,  0.5551, -0.4105],
                 

In [23]:
%timeit t_cat(trees)

29.8 µs ± 79.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.4 µs ± 86.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.0786,  1.0145, -0.3751, -1.8855],
                       [ 1.2679,  0.6253,  0.5551, -0.4105],
                       [ 2.0189, -0.1684,  0.0351, -0.0676]],
              
                      [[-0.0786,  1.0145, -0.3751, -1.8855],
                       [ 1.2679,  0.6253,  0.5551, -0.4105],
                       [ 2.0189, -0.1684,  0.0351, -0.0676]],
              
                      [[-0.0786,  1.0145, -0.3751, -1.8855],
                       [ 1.2679,  0.6253,  0.5551, -0.4105],
                       [ 2.0189, -0.1684,  0.0351, -0.0676]],
              
                      [[-0.0786,  1.0145, -0.3751, -1.8855],
                       [ 1.2679,  0.6253,  0.5551, -0.4105],
                       [ 2.0189, -0.1684,  0.0351, -0.0676]],
              
                      [[-0.0786,  1.0145, -0.3751, -1.8855],
                       [ 1.2679,  0.6253,  0.5551, -0.4105],
                       [ 2.0189, -0.1684,  0.0351, -0.0676]],

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 229 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.0786,  1.0145, -0.3751, -1.8855],
                      [ 1.2679,  0.6253,  0.5551, -0.4105],
                      [ 2.0189, -0.1684,  0.0351, -0.0676],
                      [-0.0786,  1.0145, -0.3751, -1.8855],
                      [ 1.2679,  0.6253,  0.5551, -0.4105],
                      [ 2.0189, -0.1684,  0.0351, -0.0676],
                      [-0.0786,  1.0145, -0.3751, -1.8855],
                      [ 1.2679,  0.6253,  0.5551, -0.4105],
                      [ 2.0189, -0.1684,  0.0351, -0.0676],
                      [-0.0786,  1.0145, -0.3751, -1.8855],
                      [ 1.2679,  0.6253,  0.5551, -0.4105],
                      [ 2.0189, -0.1684,  0.0351, -0.0676],
                      [-0.0786,  1.0145, -0.3751, -1.8855],
                      [ 1.2679,  0.6253,  0.5551, -0.4105],
                      [ 2.0189, -0.1684,  0.0351, -0.0676],
                      [-0.0786,  1.0145, -0.3751, -1.8855],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 260 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

346 µs ± 16.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
